In [3]:
from json_file_func import *

In [1]:
input_folder = r"C:\final_project\freqs_power_model\freqs_power_abs_ThABG_200ms_L0123\H7\ALL"
output_file = r"C:\final_project\freqs_power_model\freqs_power_abs_ThABG_200ms_L0123\H7\labels_count\labels_count_json.csv"

In [5]:
merge_json_files(input_folder, output_file)

In [22]:
import csv
import json

# Define the JSON file path
json_file_path = output_file

# Read the JSON data from the file
with open(json_file_path) as file:
    data = json.load(file)

# Extract the list of labels from the JSON data
labels = data["abstractedFeatures"]

# Extract the entity data
entities = data["entitiesData"]

# Define the output CSV file path
csv_file_path = r"C:\final_project\freqs_power_model\freqs_power_abs_ThABG_200ms_L0123\H7\labels_count\labels_count_table.csv"

# Write the data to the CSV file
with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Id"] + labels)

    # Write the data rows
    for entity in entities:
        entity_name = entity["entityName"]
        label_counts = {label: 0 for label in labels}  # Create a new dictionary for each entity

        for record in entity["records"]:
            label = record["label"]
            label_counts[label] += 1

        counts = [label_counts[label] for label in labels]
        writer.writerow([entity_name] + counts)

# Read the CSV file to check for empty columns
with open(csv_file_path, mode="r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Find the empty columns (all values are zero)
empty_columns = [index for index, column in enumerate(zip(*rows[1:])) if all(value == '0' for value in column)]

# Remove the empty columns from the data
for index in empty_columns[::-1]:
    for row in rows:
        del row[index]

# Write the updated data to the CSV file
with open(csv_file_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(rows)

print("CSV file created successfully!")



CSV file created successfully!


In [2]:
import pandas as pd

In [3]:
from df_functions import *

In [23]:
X_file_path = r"C:\final_project\freqs_power_model\freqs_power_abs_ThABG_200ms_L0123\H7\labels_count\labels_count_table.csv"
y_file_path = r"C:\final_project\H1H7 test Yonatan imputated.csv"
output_folder = r"C:\final_project\freqs_power_model\freqs_power_abs_ThABG_200ms_L0123\H7\labels_count"
train_or_test = 'H7_full'
hs_md_hsmd = ''
label = 'HDRS21_Change_CenteredByStite'

In [24]:
merge_x_and_y(X_file_path, y_file_path, output_folder, train_or_test, hs_md_hsmd,label)

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score

In [7]:
df=pd.read_csv(r"C:\final_project\freqs_power_model\freqs_power_abs_ThABG_200ms_L0123\H7\labels_count\X_H7_full__on_HDRS21_Change_CenteredByStite.csv")

In [8]:
df['subject'] = df["Id"].str[0:5]
df['HDRS21_improvement_CenteredByStite'] = df['HDRS21_Change_CenteredByStite']*(-1)
median_value = df['HDRS21_improvement_CenteredByStite'].median()


# Step 2: Create a new label column
df['above_median_HDRS21_improvement_CenteredByStite'] =  df['HDRS21_improvement_CenteredByStite'].apply(lambda x: 1.0 if x > median_value else 0.0)

In [9]:
df.head()

Id subject  Genderio  Age  HDRS21_Change_CenteredByStite  \
0  CN-02-sess1   CN-02         0   69                       6.241558   
1  CN-10-sess1   CN-10         1   43                       3.241558   
2  CN-11-sess1   CN-11         0   46                      -1.758442   
3  CN-13-sess1   CN-13         1   38                      -2.758442   
4  CN-14-sess1   CN-14         1   49                       3.584416   

   Fp1-Theta-L1  Fp1-Theta-L2  Fp1-Theta-L3  Fp1-Alpha-L1  Fp1-Alpha-L2  ...  \
0          1158           210            32          1126           228  ...   
1          1480            19             1           917           363  ...   
2          1231            36             3           859           340  ...   
3          1372            83             5          1216           200  ...   
4          1260             0             0          1193            65  ...   

   O2-Beta-L0  O2-Beta-L1  O2-Beta-L2  O2-Beta-L3  O2-Gamma-L0  O2-Gamma-L1  \
0          39        1100         218          43           61         1055   
1          89        1071         241          99           39         1094   
2           8         537         374         351           39          687   
3         579         772          94          15          422          960   
4         292         850          84          34          438          778   

   O2-Gamma-L2  O2-Gamma-L3  HDRS21_improvement_CenteredByStite  \
0          223           61                           -6.241558   
1          269           98                           -3.241558   
2          351          193                            1.758442   
3           60           18                            2.758442   
4           34           10                           -3.584416   

   above_median_HDRS21_improvement_CenteredByStite  
0                                              0.0  
1                                              0.0  
2                                              1.0  
3                                              1.0  
4                                              0.0  

[5 rows x 430 columns]

In [10]:
X = df.drop(['Id','subject','HDRS21_Change_CenteredByStite','HDRS21_improvement_CenteredByStite','above_median_HDRS21_improvement_CenteredByStite'], axis = 1)
y = df['above_median_HDRS21_improvement_CenteredByStite']

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier

In [14]:

combined_results = pd.DataFrame()  # Create an empty DataFrame to store results

accuracy_scores = []
f1_scores = []
recall_scores = []
precision_scores = []

for i in range(1, 20):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i, stratify = y)
    
    # Scale the data
    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.transform(X_test)
    
    model = XGBClassifier()
    model.fit(scaled_X_train, y_train)
    y_pred = model.predict(scaled_X_test)
    
    pred_acc = accuracy_score(y_test, y_pred)
    pred_f1 = f1_score(y_test, y_pred, average='macro')
    pred_recall = recall_score(y_test, y_pred, average='macro')
    pred_precision = precision_score(y_test, y_pred, average='macro')
    
    accuracy_scores.append(pred_acc)
    f1_scores.append(pred_f1)
    recall_scores.append(pred_recall)
    precision_scores.append(pred_precision)
    

# Calculate mean and standard deviation for each metric
mean_pred_acc= np.mean(accuracy_scores)
mean_f1_score = np.mean(f1_scores)
mean_recall_score = np.mean(recall_scores)
mean_precision_score = np.mean(precision_scores)

std_pred_acc = np.std(accuracy_scores)
std_f1_score = np.std(f1_scores)
std_recall_score = np.std(recall_scores)
std_precision_score = np.std(precision_scores)

new_scores = {'XGBoost': {'Accuracy': mean_pred_acc, 'F1': mean_f1_score, 'Recall': mean_recall_score, 'Precision': mean_precision_score}}
new_scores_table = pd.DataFrame(new_scores).T
new_scores_table['Iteration'] = 'Mean'   
combined_results = pd.concat([combined_results, new_scores_table])  # Append to combined results DataFrame

new_scores = {'XGBoost': {'Accuracy': std_pred_acc, 'F1': std_f1_score, 'Recall': std_recall_score, 'Precision': std_precision_score}}
new_scores_table = pd.DataFrame(new_scores).T
new_scores_table['Iteration'] = 'std'  
combined_results = pd.concat([combined_results, new_scores_table])  # Append to combined results DataFrame

# Reset index of combined_results DataFrame
combined_results.reset_index(inplace=True)
combined_results.rename(columns={'index': 'Model'}, inplace=True)

In [15]:
combined_results

Model  Accuracy        F1  Precision    Recall Iteration
0  XGBoost  0.555921  0.550272   0.557420  0.555921      Mean
1  XGBoost  0.112709  0.115379   0.117777  0.112709       std